iMPORTS  


In [2]:
from langchain.chat_models import init_chat_model

from dotenv import load_dotenv
import os
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent

c:\Ai_asistant\venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


Get API  Key from environment 

In [10]:
with open("KEY.txt", "r") as f:
    api_key = f.read().strip()

load_dotenv()
os.environ["OPENAI_API_KEY"] =  api_key


Set llm

In [11]:
model = init_chat_model("openai:gpt-4.1")

testing  response   

In [12]:
response = model.invoke("Why do parrots talk?")
print(response)

content='Parrots "talk" because they are exceptional vocal mimics. Here’s why and how they do it:\n\n### **Why Parrots Talk**\n1. **Social Nature:**  \n   In the wild, parrots live in flocks and need to communicate with each other for survival. Mimicking sounds is part of their social behavior—it\'s how they bond, establish hierarchy, and convey information.\n\n2. **Adaptation to Their Environment:**  \n   When kept as pets, parrots see their human owners as members of their flock. They mimic human speech and household noises to fit in and interact—the same way they would with other parrots.\n\n3. **Intelligence & Curiosity:**  \n   Parrots are highly intelligent and curious. Copying sounds and speech stimulates their minds and relieves boredom.\n\n### **How Parrots Talk**\n- **Vocal Organs:**  \n   Parrots have a specialized vocal organ called the **syrinx** (unlike human vocal cords). This allows them to produce a wide range of sounds, including mimicry of human speech patterns and t

Add memory

In [6]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

Set general instructions  to the bot  

In [ ]:
SYSTEM_PROMPT    = """
Ти асистент з фінансового аналізу.
Твої відповіді мають стосуватися лише фінансового аналізу, інвестування, бухгалтерії та фінансових метрик.
Ігноруй будь-які запити, які не пов’язані з фінансами.
"""


@tool
def get_portfolio_value(user_id: str) -> str:
    """Отримати поточну вартість інвестиційного портфеля користувача."""
    # Заглушка замысть даних з реального API
    mock_data = {"1": 125000, "2": 57800}
    value = mock_data.get(user_id, 0)
    return f"Поточна вартість портфеля користувача {user_id} становить ${value:,}."

@dataclass
class Context:
    """Контекст виконання інструментів."""
    user_id: str

@tool
def get_market_summary(runtime: ToolRuntime[Context]) -> str:
    """Отримати короткий огляд ринку на основі користувацьких даних."""
    user_id = runtime.context.user_id
    location = "Нью-Йоркська біржа" if user_id == "1" else "NASDAQ"
    return f"Основні індекси {location} сьогодні зросли на 1.2%."



@dataclass
class ResponseFormat:
    """Схема відповіді фінансового радника."""
    # Основна порада або висновок
    financial_advice: str
    # Додаткова інформація про ринок або інвестиції
    market_insight: str | None = None

In [8]:
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[ get_portfolio_value, get_market_summary],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "Di any new insights at the current time"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])



ResponseFormat(financial_advice='Якщо ви маєте інвестиції в американські акції, варто стежити за подальшими рухами ринку та аналізувати, чи укріплення індексів підтримується фундаментальними економічними даними. Для нових вкладень доцільно дочекатися підтвердження позитивного тренду.', market_insight='Основні індекси Нью-Йоркської біржі сьогодні зросли на 1.2%. Це свідчить про позитивні настрої на ринку акцій США на даний момент.')


Simple  interface for chating

In [ ]:
import tkinter as tk
from tkinter import scrolledtext

def send_message():
    user_input = entry.get()
    if user_input:
        chat_history.insert(tk.END, f"User: {user_input}\n")
        
        response = agent.invoke(
            {"messages": [{"role": "user", "content": user_input}]},
            config=config,
            context=Context(user_id="1")
        )
        
        chat_history.insert(tk.END, f"Assistant: {response['structured_response']}\n\n")
        entry.delete(0, tk.END)


root = tk.Tk()
root.title("Financial Assistant Chat")
root.geometry("600x400")

# Create chat history area
chat_history = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=70, height=20)
chat_history.pack(padx=10, pady=10)


input_frame = tk.Frame(root)
input_frame.pack(padx=10, pady=5, fill=tk.X)

entry = tk.Entry(input_frame)
entry.pack(side=tk.LEFT, fill=tk.X, expand=True)

send_button = tk.Button(input_frame, text="Send", command=send_message)
send_button.pack(side=tk.RIGHT, padx=5)

entry.bind("<Return>", lambda event: send_message())

root.mainloop()